As an ER physician, I want to determine if a patient with signs and symptoms of pneumonia should be admitted to the hospital or should receive outpatient treatment. I will support my decision using a clinical calculators known as CURB65 (https://www.mdcalc.com/curb-65-score-pneumonia-severity)

One of the main advantages of having patient information in electronic format, is that we, as clinician don't need to capture patient information more than one time. For this assignment we want to develop an application that can retrieve patient information from a EHR sandbox and calculate patient’s clinical risk

Step 1 - Define a clinical scenario with required data elements for the case: (Patient Age, last Glasgow Coma Scale, BUN in mg/dL, Respiratory Rate and Blood Pressure)

Step 2 - Identify FHIR Resources to support use case data = http://hl7.org/fhir/resourcelist.html Patient Age - http://hl7.org/fhir/patient.html Patient.birthDate minus today's date Patient Gender - Patient.contact.gender BUN in mg/dL - Observation resource - http://www.hl7.org/fhir/diagnosticreport-example-lipids.html
Blood Pressure - Observation resource - http://hl7.org/fhir/observation-vitalsigns.html
    http://www.hl7.org/fhir/observation-example-bloodpressure.html
Diabetes - Condition resource - http://hl7.org/fhir/condition.html

Step 3 - Identify data elements within each FHIR Resource definition necessary to support use case data


In [2]:
from fhirclient import client
import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta

# Define the FHIR Endpoint
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://fhirtest.uhn.ca/baseDstu3'
}
smart = client.FHIRClient(settings=settings)

# Perform a GET on patient with id of 'cf-1499698397063'
import fhirclient.models.patient as p
patient = p.Patient.read('cf-1499698397063', smart.server)

# Print the first (i.e. [0] element in the patient.name
print ('Patient\'s Name = ',patient.name[0].given[0],patient.name[0].family)

# Define variable DOB as the patient's date of birth in a string format
DOB = patient.birthDate.isostring

# Print patient's gender
gender = patient.gender
print ('Gender = ',gender)

# Print the patient's date of birth
print ('DOB = ', DOB)

# Define and print today variable
now = datetime.datetime.today()

print ('Today\'s Date = ', now)

#Define DOB2 date object
DOB2 = datetime.datetime.strptime(DOB, '%Y-%m-%d')

# Calcuate patient's age using rdelta method of the dateutil module
rdelta = relativedelta(now, DOB2)
age = rdelta.years
# print the years from the rdelta object
print ('Patient\'s age = ',age)

Patient's Name =  Curb Old
Gender =  male
DOB =  1921-06-08
Today's Date =  2017-07-21 15:02:57.442622
Patient's age =  96


In [3]:
# Confusion - GCS Observation.category - Exam
import fhirclient.models.observation as obs
search = obs.Observation.where(struct={'patient':"cf-1499698397063",'code':"9269-2"})
Glasgow_coma_score_total = search.perform_resources(smart.server)
Glasgow_flag = False
if Glasgow_coma_score_total:
    #GCS_val = LDL[0].valueQuantity.value
    #print(str(GCS_val) + " " + GCS[0].valueQuantity.unit)
    Glasgow_flag = True
    print("This patient experiences confusion")



This patient experiences confusion


In [4]:
# BUN
search = obs.Observation.where(struct={'patient':"cf-1499698397063",'code':"LP14288-2"})
BUN = search.perform_resources(smart.server)
if BUN:
    BUN_value = BUN[0].valueQuantity.value
    BUN_units = BUN[0].valueQuantity.unit
print(BUN_value)

20


In [5]:
# SBP
search = obs.Observation.where(struct={'patient':"cf-1499698397063",'code':"8480-6"})
SBP = search.perform_resources(smart.server)
if SBP:
    SBP_value = SBP[0].valueQuantity.value
SBP_value

124

In [6]:
# DBP
search = obs.Observation.where(struct={'patient':"cf-1499698397063",'code':"8462-4"})
DBP = search.perform_resources(smart.server)
if DBP:
    DBP_value = DBP[0].valueQuantity.value
DBP_value


90

In [7]:
# Resp Rate
search = obs.Observation.where(struct={'patient':"cf-1499698397063",'code':"9279-1"})
Resp = search.perform_resources(smart.server)
if Resp:
    Resp_value = Resp[0].valueQuantity.value
Resp_value


24

In [8]:
total = 0
if Glasgow_flag:
    total += 1
if BUN_units[0:2]=='mg':
    if BUN_value > 19:
        total += 1
if BUN_units[0:2]=='mm':
    if BUN_value > 7:
        total += 1
if Resp_value >= 30:
    total += 1
if SBP_value < 90 or DBP_value <= 60:
    total += 1
if age >= 65:
    total += 1
total

3

In [10]:
results_dict = {0:0.6,1:2.7,2:6.8,3:14.0,4:27.8,5:27.8}
percent = results_dict[total]
print (str(percent)+'% 30-day mortality')

14.0% 30-day mortality
